In [24]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [2]:
max_words = 1000
random_state = 4

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

In [13]:
df = pd.read_csv("../data/raw/Smart_Farming_Crop_Yield_2024.csv")
df


,farm_id,region,crop_type,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,irrigation_type,...,sowing_date,harvest_date,total_days,yield_kg_per_hectare,sensor_id,timestamp,latitude,longitude,NDVI_index,crop_disease_status
0,FARM0001,North India,Wheat,35.95,5.99,17.79,75.62,77.03,7.27,None,...,2024-01-08,2024-05-09,122,4408.07,SENS0001,2024-03-19,14.970941,82.997689,0.63,Mild
1,FARM0002,South USA,Soybean,19.74,7.24,30.18,89.91,61.13,5.67,Sprinkler,...,2024-02-04,2024-05-26,112,5389.98,SENS0002,2024-04-21,16.613022,70.869009,0.58,None
2,FARM0003,South USA,Wheat,29.32,7.16,27.37,265.43,68.87,8.23,Drip,...,2024-02-03,2024-06-26,144,2931.16,SENS0003,2024-02-28,19.503156,79.068206,0.80,Mild
3,FARM0004,Central USA,Maize,17.33,6.03,33.73,212.01,70.46,5.03,Sprinkler,...,2024-02-21,2024-07-04,134,4227.80,SENS0004,2024-05-14,31.071298,85.519998,0.44,None
4,FARM0005,Central USA,Cotton,19.37,5.92,33.86,269.09,55.73,7.93,None,...,2024-02-05,2024-05-20,105,4979.96,SENS0005,2024-04-13,16.568540,81.691720,0.84,Severe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,FARM0496,Central USA,Rice,42.85,6.70,30.85,52.35,79.58,7.25,Manual,...,2024-01-16,2024-06-02,138,4251.40,SENS0496,2024-05-08,30.386623,76.147700,0.59,Mild
496,FARM0497,North India,Soybean,34.22,6.75,17.46,256.23,45.14,5.78,None,...,2024-01-01,2024-04-14,104,3708.54,SENS0497,2024-01-19,18.832748,75.736924,0.85,Severe
497,FARM0498,North India,Cotton,15.93,5.72,17.03,288.96,57.87,7.69,Drip,...,2024-01-02,2024-05-09,128,2604.41,SENS0498,2024-04-20,23.262016,81.992230,0.71,Mild
498,FARM0499,Central USA,Soybean,38.61,6.20,17.08,279.06,73.09,9.60,Drip,...,2024-01-25,2024-06-04,131,2586.36,SENS0499,2024-03-02,19.764989,84.426869,0.77,Severe


In [31]:
df.drop(columns=["farm_id", "sensor_id"], inplace=True) #clear the id columns
df.to_csv("../data/raw/Smart_Farming_Crop_Yield_2024.csv")
df

,region,crop_type,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,irrigation_type,fertilizer_type,...,yield_kg_per_hectare,timestamp,latitude,longitude,NDVI_index,crop_disease_status,region_le,croptype_le,cropdisease_le,irrigation_type_le
0,North India,Wheat,35.95,5.99,17.79,75.62,77.03,7.27,None,Organic,...,4408.07,2024-03-19,14.970941,82.997689,0.63,Mild,2,4,0,2
1,South USA,Soybean,19.74,7.24,30.18,89.91,61.13,5.67,Sprinkler,Inorganic,...,5389.98,2024-04-21,16.613022,70.869009,0.58,None,4,3,2,3
2,South USA,Wheat,29.32,7.16,27.37,265.43,68.87,8.23,Drip,Mixed,...,2931.16,2024-02-28,19.503156,79.068206,0.80,Mild,4,4,0,0
3,Central USA,Maize,17.33,6.03,33.73,212.01,70.46,5.03,Sprinkler,Organic,...,4227.80,2024-05-14,31.071298,85.519998,0.44,None,0,1,2,3
4,Central USA,Cotton,19.37,5.92,33.86,269.09,55.73,7.93,None,Mixed,...,4979.96,2024-04-13,16.568540,81.691720,0.84,Severe,0,0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Central USA,Rice,42.85,6.70,30.85,52.35,79.58,7.25,Manual,Mixed,...,4251.40,2024-05-08,30.386623,76.147700,0.59,Mild,0,2,0,1
496,North India,Soybean,34.22,6.75,17.46,256.23,45.14,5.78,None,Organic,...,3708.54,2024-01-19,18.832748,75.736924,0.85,Severe,2,3,3,2
497,North India,Cotton,15.93,5.72,17.03,288.96,57.87,7.69,Drip,Inorganic,...,2604.41,2024-04-20,23.262016,81.992230,0.71,Mild,2,0,0,0
498,Central USA,Soybean,38.61,6.20,17.08,279.06,73.09,9.60,Drip,Organic,...,2586.36,2024-03-02,19.764989,84.426869,0.77,Severe,0,3,3,0


In [ ]:
def preprocess(text, stop_words, punctuation_marks, morph): #for the future
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
      if token not in punctuation_marks:
        lemma = morph.lemmatize(token)
        if lemma not in stop_words:
          preprocessed_text.append(lemma)
    return preprocessed_text


In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '/', '@', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '"']
morph = WordNetLemmatizer() # for the future

In [19]:
stop_words = stopwords.words("english")

In [36]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["fertilizer_type_le"] = le.fit_transform(df["fertilizer_type"])

df.head(8)

df.to_csv("../data/preprocessed/Smart_Farming_Crop_Yield_2024.csv")

In [46]:
new_df.head()
new_df.to_csv("../data/preprocessed/Smart_Farming_Crop_Yield_2024.csv")

In [ ]:
new_df.head() #i deleted all future features, to avoid the problems during the train

,soil_moisture_%,soil_pH,pesticide_usage_ml,sowing_date,yield_kg_per_hectare,latitude,longitude,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le
0,35.95,5.99,6.34,2024-01-08,4408.07,14.970941,82.997689,2,4,0,2,2
1,19.74,7.24,9.60,2024-02-04,5389.98,16.613022,70.869009,4,3,2,3,0
2,29.32,7.16,15.26,2024-02-03,2931.16,19.503156,79.068206,4,4,0,0,1
3,17.33,6.03,25.80,2024-02-21,4227.80,31.071298,85.519998,0,1,2,3,2
4,19.37,5.92,25.65,2024-02-05,4979.96,16.568540,81.691720,0,0,3,2,1


In [48]:
new_df["sowing_month"] = pd.to_datetime(df["sowing_date"]).dt.month
new_df["sowing_season"] = pd.to_datetime(df["sowing_date"]).dt.month % 12 // 3 + 1
new_df.head()


,soil_moisture_%,soil_pH,pesticide_usage_ml,sowing_date,yield_kg_per_hectare,latitude,longitude,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le,sowing_month,sowing_season
0,35.95,5.99,6.34,2024-01-08,4408.07,14.970941,82.997689,2,4,0,2,2,1,1
1,19.74,7.24,9.60,2024-02-04,5389.98,16.613022,70.869009,4,3,2,3,0,2,1
2,29.32,7.16,15.26,2024-02-03,2931.16,19.503156,79.068206,4,4,0,0,1,2,1
3,17.33,6.03,25.80,2024-02-21,4227.80,31.071298,85.519998,0,1,2,3,2,2,1
4,19.37,5.92,25.65,2024-02-05,4979.96,16.568540,81.691720,0,0,3,2,1,2,1


In [50]:
new_df.drop(columns=["sowing_date"], inplace=True)
new_df.head()

,soil_moisture_%,soil_pH,pesticide_usage_ml,yield_kg_per_hectare,latitude,longitude,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le,sowing_month,sowing_season
0,35.95,5.99,6.34,4408.07,14.970941,82.997689,2,4,0,2,2,1,1
1,19.74,7.24,9.60,5389.98,16.613022,70.869009,4,3,2,3,0,2,1
2,29.32,7.16,15.26,2931.16,19.503156,79.068206,4,4,0,0,1,2,1
3,17.33,6.03,25.80,4227.80,31.071298,85.519998,0,1,2,3,2,2,1
4,19.37,5.92,25.65,4979.96,16.568540,81.691720,0,0,3,2,1,2,1


In [51]:
new_df.isnull().sum()
new_df.describe()

,soil_moisture_%,soil_pH,pesticide_usage_ml,yield_kg_per_hectare,latitude,longitude,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le,sowing_month,sowing_season
count,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.000000
mean,26.750140,6.523980,26.586980,4032.92694,22.442473,80.392248,1.908000,1.934000,1.542000,1.56200,0.998000,2.062000,1.374000
std,10.150053,0.585558,13.202429,1174.43304,7.283492,5.910664,1.419707,1.424677,1.132608,1.08467,0.816903,0.826755,0.484348
min,10.160000,5.510000,5.050000,2023.56000,10.004243,70.020021,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000
25%,17.890000,6.030000,14.945000,2994.82000,16.263202,75.374713,1.000000,1.000000,0.750000,1.00000,0.000000,1.000000,1.000000
50%,25.855000,6.530000,25.980000,4071.69000,21.981743,80.650284,2.000000,2.000000,2.000000,2.00000,1.000000,2.000000,1.000000
75%,36.022500,7.040000,38.005000,5062.11000,28.528948,85.654629,3.000000,3.000000,3.000000,2.00000,2.000000,3.000000,2.000000
max,44.980000,7.500000,49.940000,5998.29000,34.981531,89.991901,4.000000,4.000000,3.000000,3.00000,2.000000,3.000000,2.000000


In [ ]:
y = new_df["yield_kg_per_hectare"] #Our target to predict 
X = new_df.drop(columns=["yield_kg_per_hectare"]) #Features
y.head()
X.head()

,soil_moisture_%,soil_pH,pesticide_usage_ml,latitude,longitude,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le,sowing_month,sowing_season
0,35.95,5.99,6.34,14.970941,82.997689,2,4,0,2,2,1,1
1,19.74,7.24,9.60,16.613022,70.869009,4,3,2,3,0,2,1
2,29.32,7.16,15.26,19.503156,79.068206,4,4,0,0,1,2,1
3,17.33,6.03,25.80,31.071298,85.519998,0,1,2,3,2,2,1
4,19.37,5.92,25.65,16.568540,81.691720,0,0,3,2,1,2,1


In [53]:
y.head()

0    4408.07
1    5389.98
2    2931.16
3    4227.80
4    4979.96
Name: yield_kg_per_hectare, dtype: float64

In [54]:
X.head()

,soil_moisture_%,soil_pH,pesticide_usage_ml,latitude,longitude,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le,sowing_month,sowing_season
0,35.95,5.99,6.34,14.970941,82.997689,2,4,0,2,2,1,1
1,19.74,7.24,9.60,16.613022,70.869009,4,3,2,3,0,2,1
2,29.32,7.16,15.26,19.503156,79.068206,4,4,0,0,1,2,1
3,17.33,6.03,25.80,31.071298,85.519998,0,1,2,3,2,2,1
4,19.37,5.92,25.65,16.568540,81.691720,0,0,3,2,1,2,1


In [57]:
X.to_csv("../data/preprocessed/trainig_data.csv")
y.to_csv("../data/preprocessed/target_data.csv")